# Australian Wildfires Dashboard

## Components of Dashboard and Expected layout

### Components of the Dashboard

1. Select Region
2. Select Year
3. Divison to display
    - Pie Chart to display Monthly Average Estimated Fire Area for the selected Regions in the selected Year
    - Bar Chart to display Monthly Average Count of Pixels for Presumed Vegetation Fires for the selected Regions in the selected Year

### Expected Layout

![Final Dashboard](https://raw.githubusercontent.com/sduzair/python/data_vis_dashboard_wildfire_activities_australia/data_vis_dashboard_wildfire_activities_australia/practice_ass_layout.png)

### Requirements to create the expected result

- A dropdown menu: For choosing year
- A radioitem for choosing the Region
- The layout will be designed as follows:
- An outer division with two inner divisions (as shown in the expected layout)
- One of the inner divisions will have information about the radioitem and dropdown (which are the input) and the other one is for adding graphs(the 2 output graphs).
- Callback function to compute data, create graph and return to the layout.

### To do:

1. Import required libraries and read the dataset
2. Create an application layout
3. Add title to the dashboard using HTML H1 component
4. Add a radioitem using dcc.RaioItems and dropdown using dcc.dropdown
5. Add the pie chart and bar chart core graph components.
6. Run the app

## Dependencies

```bash
cat python_version.txt
Python 3.10.12


In [ ]:
!python --version

In [ ]:
%pip install -q pandas==2.0.3 plotly==5.15.0 dash==2.16.1 dash-bootstrap-components

## Imports

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from dash import Dash, html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc
import warnings
%config InlineBackend.figure_format = 'retina'

warnings.filterwarnings('ignore')

## Loading data

In [ ]:
# Read the wildfire data into pandas dataframe
df =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv')

## Data wrangling

In [ ]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df: pd.DataFrame):
    # Change column type to datetime64[ns] for column: 'Date'
    df = df.astype({'Date': 'datetime64[ns]'})
    # Created column 'Year' from formula
    df['Year'] = df['Date'].dt.year
    # Created column 'Month' from formula
    df['Month'] = df['Date'].dt.month
    # Change column type to category for column: 'Month'
    df = df.astype({'Month': 'category'})
    # Change column type to category for column: 'Replaced'
    df = df.astype({'Replaced': 'category'})
    # Change column type to category for column: 'Region'
    df = df.astype({'Region': 'category'})
    # Change column type to int16 for column: 'Count'
    df = df.astype({'Count': 'int16'})
    return df

df_clean = clean_data(df.copy())
df_clean.head()

In [ ]:
def display_mem_usage(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    mem_usage = pd.DataFrame({'df': df.memory_usage(deep=True), 'df_clean': df_clean.memory_usage(deep=True)})
    total = mem_usage.sum().to_frame().T
    total.index = ['Total']

    return pd.concat([total, mem_usage]).T

display_mem_usage(df, df_clean)

## Pie Chart to display Monthly Average Estimated Fire Area for the selected Regions in the selected Year

In [ ]:
# Pie Chart to display Monthly Average Estimated Fire Area for the selected Regions in the selected Year
month_map = {1:'Jan', 2:'Feb', 3:'March', 4:'April', 5:'May', 6:'June', 7:'July', 8:'August', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}

def avg_efa_by_month_pie(df_avg_efa_by_month: pd.DataFrame, year=2005, region='NSW') -> go.Figure:
  df_for_year_reg = df_clean[(df_avg_efa_by_month['Year'] == year) & (df_avg_efa_by_month['Region'] == region)]
  df_avg_efa_by_month = df_for_year_reg.groupby('Month')['Estimated_fire_area'].mean().reset_index()
  df_avg_efa_by_month['Month label'] = df_avg_efa_by_month['Month'].map(month_map)

  return px.pie(
      data_frame=df_avg_efa_by_month,
      names='Month label',
      values='Estimated_fire_area',
      title=f'{region}: Monthly Average Estimated Fire Area in the Year {year}')

avg_efa_by_month_pie(df_clean)


## Bar Chart to display Monthly Average Count of Pixels for Presumed Vegetation Fires for the selected Regions in the selected Year

In [ ]:
# Bar Chart to display Monthly Average Count of Pixels for Presumed Vegetation Fires for the selected Regions in the selected Year
def avg_count_by_month_bar(df: pd.DataFrame, year=2005, region='NSW') -> go.Figure:
  df_for_year_reg = df[(df['Year'] == year) & (df['Region'] == region)]
  df_avg_count_by_month = df_for_year_reg.groupby('Month')['Count'].mean().reset_index()
  df_avg_count_by_month['Month label'] = df_avg_count_by_month['Month'].map(month_map)

  return px.bar(
      data_frame=df_avg_count_by_month,
      x='Month label',
      y='Count',
      title=f'{region}: Monthly Average Count of Pixels for Presumed Vegetation Fires in the year {year}'
  )

avg_count_by_month_bar(df_clean)

## Dash App

In [ ]:
# Initialize the app - incorporate a Dash Bootstrap theme
external_stylesheets = [dbc.themes.CERULEAN]
app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = dbc.Container([
  dbc.Row([html.H1('Austrailan Wildfires Dashboard', className="text-primary text-center fs-3")]),
  dbc.Row([
    dbc.Label("Region"),
    dbc.RadioItems(options=[{"label": x, "value": x} for x in df_clean['Region'].unique()], value='NSW', inline=True, id='radio-region')
  ]),
  dbc.Row([
    dbc.Label("Year"),
    dcc.Dropdown(df_clean['Year'].unique(), value='2005', id='dropdown-year'),
  ]),
  dbc.Row([
    dbc.Col([dcc.Graph(figure={}, id='fig-pie')], width=6),
    dbc.Col([dcc.Graph(figure={}, id='fig-bar')], width=6),
  ]),
], fluid=True)

@callback(
  Output('fig-pie', 'figure'),
  Output('fig-bar', 'figure'),
  Input('dropdown-year', 'value'),
  Input('radio-region', 'value')
)
def update_graph(year, region):
  df_pie = avg_efa_by_month_pie(df_clean, year, region)
  df_bar = avg_count_by_month_bar(df_clean, year, region)
  return df_pie, df_bar

if __name__ == '__main__':
  app.run(debug=True)